In [2]:
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
#Checking data types of train
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [6]:
#Cecking for missing values
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
from sklearn.impute import KNNImputer
knn=KNNImputer()

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()

In [10]:
#Function for dropping uncessary columns,using knn imputer for numerican missing values and 
#mode for categorical columns
def clean(df):
    df=df.drop(columns = ['PassengerId','Name','Cabin'])
    num = df.select_dtypes(exclude='object').columns
    cat = df.select_dtypes(include='object').columns
    df[num]=knn.fit_transform(df[num])   
    
    for i in cat:
        df[i].fillna(df[i].mode()[0],inplace = True)
        df[i] = le.fit_transform(df[i])
    df['CryoSleep'] = df['CryoSleep'].astype(int)
    df['VIP'] = df['VIP'].astype(int)
    if 'Transported' in df.columns:
        df['Transported'] = df['Transported'].astype(int)
    return df

#function for scaleing the data using Robusrt scaler
def scale(df):
    cols = df.columns
    df = rs.fit_transform(df)
    df = pd.DataFrame(df,columns = cols)
    return df

In [11]:
df_clean=clean(df)

In [12]:
df_sclaed = scale(df_clean)

In [13]:
test_clean = clean(test)

In [14]:
test_scaled = scale(test_clean)

In [15]:
test_scaled.head(3)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,1.0,0.0,0.055556,0.0,0.0,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.0,-0.388889,0.0,0.0,0.105882,0.0,50.410714,0.0
2,1.0,1.0,-2.0,0.277778,0.0,0.0,0.000000,0.0,0.000000,0.0


In [16]:
X = df_sclaed.drop(columns=["Transported"],axis = 1)
y = df['Transported']

In [17]:
X.head(3)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,1.0,0.0,0.0,0.705882,0.0,0.000000,0.0000,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,-0.176471,0.0,2.137255,0.1125,0.833333,8.854839,0.897959
2,1.0,0.0,0.0,1.823529,1.0,0.843137,44.7000,0.000000,108.306452,1.000000


In [81]:
from sklearn.model_selection import train_test_split

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
lr = LogisticRegression()

In [105]:
lr.fit(X_train,y_train)

LogisticRegression()

In [106]:
train_pred = lr.predict(X_train)
test_pred = lr.predict(X_test)

In [107]:
from sklearn.metrics import accuracy_score

In [108]:
print("Train Accuracy :: ",accuracy_score(y_train,train_pred))
print("Test Accuracy :: ",accuracy_score(y_test,test_pred))

Train Accuracy ::  0.7911993097497843
Test Accuracy ::  0.7717078780908568


In [109]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [110]:
train_pred = rfc.predict(X_train)
test_pred = rfc.predict(X_test)

In [111]:
print("Train Accuracy :: ",accuracy_score(y_train,train_pred))
print("Test Accuracy :: ",accuracy_score(y_test,test_pred))

Train Accuracy ::  0.9344262295081968
Test Accuracy ::  0.7855089131684876


In [1]:
#Random forest is underfitting , thats why doing hyperparameter tuning

In [112]:
from sklearn.model_selection import GridSearchCV

In [113]:
param_grid = {
    #'n_estimators': [50, 100, 200],  
    'max_depth': [ 12, 13, 9,10 ,11], 
    'min_samples_split': [2, 5, 6],  
    'min_samples_leaf': [1, 2, 4,5,6], 
    #'max_features': ['auto', 'sqrt'], 
     #'bootstrap': [True, False]  
  
}

In [115]:
gv = GridSearchCV(estimator=rfc,param_grid=param_grid,cv=3)

In [116]:
gv.fit(X,y)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [12, 13, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 4, 5, 6],
                         'min_samples_split': [2, 5, 6]})

In [117]:
gv.best_params_

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [118]:
rfc  = RandomForestClassifier(max_depth=10,min_samples_leaf=1,min_samples_split=2)

In [119]:
rfc.fit(X_train,y_train)

RandomForestClassifier(max_depth=10)

In [120]:
train_pred = rfc.predict(X_train)
test_pred = rfc.predict(X_test)

In [121]:
print("Train Accuracy :: ",accuracy_score(y_train,train_pred))
print("Test Accuracy :: ",accuracy_score(y_test,test_pred))

Train Accuracy ::  0.8527466206499856
Test Accuracy ::  0.7918343875790684


In [130]:
#hyper parameter tuning of Logistic regression
param_grid1 = {
    'penalty': ['l1', 'l2'], 
    'C': [0.001, 0.01, 0.1, 1.0, 10.0],  
    'solver': ['liblinear', 'saga'],  
    'max_iter': [15,25,50,100]  
}

In [131]:
gv = GridSearchCV(estimator=lr,param_grid=param_grid1,cv=3)

In [132]:
gv.fit(X,y)

GridSearchCV(cv=3,
             estimator=LogisticRegression(C=0.001, max_iter=15, penalty='l1',
                                          solver='saga'),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10.0],
                         'max_iter': [15, 25, 50, 100, 15],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [133]:
gv.best_params_

{'C': 0.001, 'max_iter': 15, 'penalty': 'l1', 'solver': 'saga'}

In [126]:
lr = LogisticRegression(C=0.001 , max_iter=15 , penalty = 'l1' , solver = 'saga')

In [127]:
lr.fit(X_train,y_train)

LogisticRegression(C=0.001, max_iter=15, penalty='l1', solver='saga')

In [128]:
train_pred = lr.predict(X_train)
test_pred = lr.predict(X_test)

In [129]:
print("Train Accuracy :: ",accuracy_score(y_train,train_pred))
print("Test Accuracy :: ",accuracy_score(y_test,test_pred))

Train Accuracy ::  0.7956571757262008
Test Accuracy ::  0.7843588269120184


In [ ]:
#after hyperparameter tuning of both,using logistic regression for final prediction

In [73]:
pred = lr.predict(test_scaled)

In [74]:
sub=pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Transported':pred
})

In [75]:
sub.to_csv("submission.csv",index=False)